In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE, RandomOverSampler

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
train_data = pd.read_csv('UNSWNB15_dataset/UNSW_NB15_training-set.csv')
test_data = pd.read_csv('UNSWNB15_dataset/UNSW_NB15_testing-set.csv')
train_data[train_data['service']!='-']
cols_to_transform = ['proto', 'service', 'state']
#train_data = pd.get_dummies(data = train_data, columns = cols_to_transform)
#test_data = pd.get_dummies(data = test_data, columns = cols_to_transform)
train_data.dtypes
#train_data[['label','attack_cat']][train_data['attack_cat']!='Dos'].head(10000)
obj_df = train_data.select_dtypes(include=['object']).copy()
for col in cols_to_transform:
    train_data[col]= train_data[col].astype('category').cat.codes
    test_data[col]= test_data[col].astype('category').cat.codes
train_data_ = train_data.drop(['id','attack_cat'], axis=1)
test_data_ = test_data.drop(['id','attack_cat'], axis=1)
train_data_ = train_data_.as_matrix(columns = None)
test_data_ = test_data_.as_matrix(columns = None)
test_data_.shape
train_data

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.000011,117,0,4,2,0,496,0,9.090909e+04,...,1,2,0,0,0,1,2,0,Normal,0
1,2,0.000008,117,0,4,2,0,1762,0,1.250000e+05,...,1,2,0,0,0,1,2,0,Normal,0
2,3,0.000005,117,0,4,2,0,1068,0,2.000000e+05,...,1,3,0,0,0,1,3,0,Normal,0
3,4,0.000006,117,0,4,2,0,900,0,1.666667e+05,...,1,3,0,0,0,2,3,0,Normal,0
4,5,0.000010,117,0,4,2,0,2126,0,1.000000e+05,...,1,3,0,0,0,2,3,0,Normal,0
5,6,0.000003,117,0,4,2,0,784,0,3.333333e+05,...,1,2,0,0,0,2,2,0,Normal,0
6,7,0.000006,117,0,4,2,0,1960,0,1.666667e+05,...,1,2,0,0,0,2,2,0,Normal,0
7,8,0.000028,117,0,4,2,0,1384,0,3.571429e+04,...,1,3,0,0,0,1,3,0,Normal,0
8,9,0.000000,6,0,4,1,0,46,0,0.000000e+00,...,2,2,0,0,0,2,2,1,Normal,0
9,10,0.000000,6,0,4,1,0,46,0,0.000000e+00,...,2,2,0,0,0,2,2,1,Normal,0


In [3]:
x_train= train_data_[:,:-1]
y_train = train_data_[:,-1]
x_test = test_data_[:,:-1]
y_test = test_data_[:,-1]
x_min = []
x_max = []
for i in range(x_train.shape[1]):
    x_min.append(np.min(x_train[:,i]))
    x_max.append(np.max(x_train[:,i]))
x_max = np.array(x_max)
x_min = np.array(x_min)
x_train = (x_train - x_min)/(x_max-x_min)
x_test = (x_test - x_min)/(x_max-x_min)
x_train,y_train

(array([[1.83333367e-07, 9.00000000e-01, 0.00000000e+00, ...,
         0.00000000e+00, 1.63934426e-02, 0.00000000e+00],
        [1.33333358e-07, 9.00000000e-01, 0.00000000e+00, ...,
         0.00000000e+00, 1.63934426e-02, 0.00000000e+00],
        [8.33333486e-08, 9.00000000e-01, 0.00000000e+00, ...,
         0.00000000e+00, 3.27868852e-02, 0.00000000e+00],
        ...,
        [0.00000000e+00, 4.61538462e-02, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
        [0.00000000e+00, 4.61538462e-02, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
        [1.50000028e-07, 9.00000000e-01, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]]),
 array([0., 0., 0., ..., 0., 0., 0.]))

In [4]:
# Number of Epochs
dim_size = 40
epochs = 200
num_seq = 30
# Batch Size
batch_size =200
# RNN Size
rnn_size =50
# Learning Rate
learning_rate = 0.0005
num_layers = 4
# Show stats for every n number of batches
show_every_n_batches = 100
keep_prob = 0.6
relu_size = 20
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

In [5]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components = dim_size)
x_reduced = svd.fit_transform(x_train)


In [6]:
x_test

array([[2.02463370e-03, 8.69230769e-01, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.08317020e-02, 8.69230769e-01, 0.00000000e+00, ...,
        0.00000000e+00, 8.19672131e-02, 0.00000000e+00],
       [2.70521550e-02, 8.69230769e-01, 0.00000000e+00, ...,
        1.69491525e-02, 8.19672131e-02, 0.00000000e+00],
       ...,
       [1.50000028e-07, 9.15384615e-01, 1.66666667e-01, ...,
        3.38983051e-02, 1.80327869e-01, 0.00000000e+00],
       [1.50000028e-07, 9.15384615e-01, 1.66666667e-01, ...,
        4.91525424e-01, 4.75409836e-01, 0.00000000e+00],
       [1.50000028e-07, 9.15384615e-01, 1.66666667e-01, ...,
        4.91525424e-01, 4.75409836e-01, 0.00000000e+00]])

In [7]:
x_test_reduced = svd.transform(x_test)

In [8]:
x_train.shape

(82332, 42)

In [9]:
def resample(arr,y, num_seq):
    
    new_arr = np.zeros((arr.shape[0],num_seq,arr.shape[1]))
    new_y = np.zeros(arr.shape[0])
    for i in range(arr.shape[0]-num_seq):
        for j in range(num_seq):
            new_arr[i,j,:] = arr[i+j,:]
        new_y[i] = y[i+num_seq-1]
    x_resampled ,y_resampled = SMOTE().fit_sample(np.reshape(new_arr,[-1,num_seq*arr.shape[1]]), new_y)
    x_resampled = np.reshape(x_resampled,[-1,num_seq,arr.shape[1]])
    return x_resampled,y_resampled


In [10]:
x_resampled = []
y_resampled = []
for i in range(3):
    x_resampledi, y_resampledi = resample(x_reduced[i*25000:25000*(i+1)], y_train[i*25000:25000*(i+1)],num_seq)
    x_resampled.append(x_resampledi)
    y_resampled.append(y_resampledi)
#x_resampledj,y_resampledj = resample(x_reduced[75000:], y_train[75000:],num_seq)
#x_resampled.append(x_resampledj)
#y_resampled.append(y_resampledj)
x_resampled = np.array(np.concatenate(x_resampled, axis = 0))    
y_resampled = np.array(np.concatenate(y_resampled, axis = 0))
from random import shuffle
ind_list = [i for i in range(x_resampled.shape[0])]
shuffle(ind_list)
x_train_ = x_resampled[ind_list]
y_train_ = y_resampled[ind_list]

In [11]:
x_valid = x_train_[-40000:]
y_valid = y_train_[-40000:]
train_x = x_train_[:-40000]
train_y = y_train_[:-40000]
x_train.shape
np.mean(y_valid)

0.50065

In [12]:
x_train

array([[1.83333367e-07, 9.00000000e-01, 0.00000000e+00, ...,
        0.00000000e+00, 1.63934426e-02, 0.00000000e+00],
       [1.33333358e-07, 9.00000000e-01, 0.00000000e+00, ...,
        0.00000000e+00, 1.63934426e-02, 0.00000000e+00],
       [8.33333486e-08, 9.00000000e-01, 0.00000000e+00, ...,
        0.00000000e+00, 3.27868852e-02, 0.00000000e+00],
       ...,
       [0.00000000e+00, 4.61538462e-02, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 4.61538462e-02, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [1.50000028e-07, 9.00000000e-01, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [13]:
def get_batches(arr,y, batch_size):
    n_batches = arr.shape[0]//batch_size
    for i in range(0,n_batches*batch_size,batch_size):
        x = arr[i:i+batch_size,:,:]
        y_ = y[i:i+batch_size]
        yield x,y_
x,y = next(get_batches(train_x, train_y, batch_size))
y.shape

(200,)

In [14]:
def get_test_batches(arr,y, batch_size, num_seq=num_seq):
    new_arr = np.zeros((arr.shape[0],num_seq,arr.shape[1]))
    new_y = np.zeros(arr.shape[0])
    for i in range(arr.shape[0]-num_seq):
        for j in range(num_seq):
            new_arr[i,j,:] = arr[i+j,:]
        new_y[i] = y[i+num_seq-1]
    n_batches = arr.shape[0]//batch_size
    for i in range(0,n_batches*batch_size,batch_size):
        x = new_arr[i:i+batch_size,:,:]
        y = new_y[i:i+batch_size]
        yield x,y
next(get_test_batches(x_test_reduced, y_test,100))       

(array([[[ 2.11737297e+00, -4.54734873e-01, -4.38193762e-01, ...,
          -4.62860741e-04, -3.25086674e-04, -7.60591623e-05],
         [ 1.99336642e+00, -7.64171994e-01,  3.06655465e-01, ...,
          -2.08592802e-03,  9.76640742e-04, -4.61669324e-04],
         [ 1.97200757e+00, -8.73143555e-01,  1.50674535e-01, ...,
          -1.16913444e-03,  7.29940067e-04, -3.99962590e-04],
         ...,
         [ 2.26161161e+00, -4.95562996e-01, -3.61075481e-01, ...,
          -2.80127013e-04, -9.11095676e-05, -1.20288109e-04],
         [ 2.32103268e+00, -5.36507154e-01, -2.76528765e-01, ...,
          -2.97933200e-04,  1.34340923e-04, -1.90675145e-04],
         [ 1.09464340e+00,  9.02895936e-01, -2.69959244e-01, ...,
          -2.20763896e-04,  2.86660593e-04, -8.28627084e-05]],
 
        [[ 1.99336642e+00, -7.64171994e-01,  3.06655465e-01, ...,
          -2.08592802e-03,  9.76640742e-04, -4.61669324e-04],
         [ 1.97200757e+00, -8.73143555e-01,  1.50674535e-01, ...,
          -1.16913444

In [15]:
def build_inputs(batch_size, num_inputs = 100 ):
    inputs = tf.placeholder(tf.float32, [None,num_inputs,dim_size], name = 'inputs')
    outputs = tf.placeholder(tf.int32, [None,],name = 'targets')
    keep_prob = tf.placeholder(tf.float32, name = 'keep_prob')
    return inputs, outputs, keep_prob

In [16]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' Build LSTM cell.
    
        Arguments
        ---------
        keep_prob: Scalar tensor (tf.placeholder) for the dropout keep probability
        lstm_size: Size of the hidden layers in the LSTM cells
        num_layers: Number of LSTM layers
        batch_size: Batch size

    '''
    ### Build the LSTM Cell
    
    def build_cell(lstm_size, keep_prob):
        # Use a basic LSTM cell
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        
        # Add dropout to the cell
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)
    return cell, initial_state   

In [17]:
def build_output(lstm_output, in_size, out_size, relu_size = 100):
    out = tf.contrib.layers.fully_connected(lstm_output, relu_size, activation_fn = tf.nn.relu)
    out = tf.layers.batch_normalization(out)
    logits = tf.contrib.layers.fully_connected(out,out_size, activation_fn=None)
    #logits = tf.reshape(logits, (-1,10,3, out_size))
    return logits


In [18]:
def build_loss(logits, targets,vocab_size):
    ''' Calculate the loss from the logits and the targets.
    
        Arguments
        ---------
        logits: Logits from final fully connected layer
        targets: Targets for supervised learning
        lstm_size: Number of LSTM hidden units
        num_classes: Number of classes in targets
        
    '''
    
      # One-hot encode targets and reshape to match logits, one row per batch_size per step
    #logits = tf.reshape(logits,[-1,vocab_size])
    y_one_hot = tf.one_hot(targets, vocab_size)
    #y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
    #y_reshaped = tf.reduce_sum(y_one_hot, axis = 1)
    # Softmax cross entropy loss
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_one_hot)
    loss = tf.reduce_mean(loss)
    return loss, y_one_hot

In [19]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Build optmizer for training, using gradient clipping.
    
        Arguments:
        loss: Network loss
        learning_rate: Learning rate for optimizer
    
    '''
    
    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

In [20]:
class RNN:
    
    def __init__(self, vocab_size, relu_size, batch_size=64,
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5):
    
        # When we're using this network for sampling later, we'll be passing in
        # one character at a time, so providing an option for that

        tf.reset_default_graph()
        
        # Build the input placeholder tensors
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_inputs = num_seq )

        # Build the LSTM cell
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        ### Run the data through the RNN layers
        # First, one-hot encode the input tokens
       
        
        # Run each sequence step through the RNN and collect the outputs
        outputs, state = tf.nn.dynamic_rnn(cell, self.inputs, initial_state=self.initial_state)
        self.final_state = state
        
        # Get softmax predictions and logits
        self.logits = build_output(outputs, lstm_size, vocab_size, relu_size = relu_size)[:,-1]
        
        # Loss and optimizer (with gradient clipping)
        self.loss, self.y_reshaped = build_loss(self.logits, self.targets, vocab_size)
        self.pred = tf.argmax(self.logits,1)
        #_,self.pred = tf.nn.top_k(self.logits,3)
        correct_pred = tf.equal(self.pred, tf.argmax(self.y_reshaped,1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

In [21]:
save_every_n = 200

model = RNN(2,relu_size, batch_size=batch_size,
                lstm_size=rnn_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    for e in range(epochs):
        train_loss = []
        
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(train_x, train_y,batch_size):
            
            counter += 1
            
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            train_loss.append(batch_loss)
            #if(counter%show_every_n_batches ==0):
                #print('Epoch: {}/{}... '.format(e+1, epochs),
                  #'Training Step: {}... '.format(counter),
                    #'Training loss: {:.4f}... '.format(batch_loss))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, num_layers))
        val_loss = []
        val_state = sess.run(model.initial_state)
        for x, y in get_batches(x_valid, y_valid, batch_size):
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: 1,
                    model.initial_state: val_state}
            batch_loss, val_state = sess.run([model.loss, 
                                                 model.final_state], 
                                                 feed_dict=feed)
            
                                                 
            val_loss.append(batch_loss)
            
        print("epoch {}: Training loss : {:.6f}".format(e, np.mean(train_loss)))
        print("Validation loss : {:.6f}".format(np.mean(val_loss)))
        
    val_acc = []
    val_pred =[]
    val_y = []
    val_logits = []
    val_state = sess.run(model.initial_state)
    for x, y in get_batches(x_valid, y_valid,batch_size):
        feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: 1,
                    model.initial_state: val_state}
        val_state,batch_pred,batch_acc,batch_logits = sess.run([model.final_state,model.pred,model.accuracy, model.logits], feed_dict = feed)
        val_acc.append(batch_acc)
        val_y.append(y)
        val_pred.append(batch_pred)
        val_logits.append(batch_logits)
    train_pred =[]
    train_label = []
    train_acc = []
    train_logits = []
    train_state = sess.run(model.initial_state)
    for x, y in get_batches(train_x, train_y,batch_size):
        feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: 1,
                    model.initial_state: train_state}
        train_state,batch_pred,batch_acc, batch_logits = sess.run([model.final_state,model.pred,model.accuracy, model.logits], feed_dict = feed)
        train_acc.append(batch_acc)
        train_label.append(y)
        train_pred.append(batch_pred)
        train_logits.append(batch_logits)
    print("Training accuracy : {:.2f}".format(100*np.mean(train_acc)))    
    print("validation accuracy : {:.2f}".format(100*np.mean(val_acc)))
    test_acc = []
    test_pred =[]
    test_y = []
    test_logits = []
    test_state = sess.run(model.initial_state)
    for x, y in get_test_batches(x_test_reduced, y_test,batch_size,num_seq):
        feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: 1,
                    model.initial_state: val_state}
        test_state,batch_pred,batch_acc, batch_logits = sess.run([model.final_state,model.pred,model.accuracy,model.logits], feed_dict = feed)
        test_acc.append(batch_acc)
        test_y.append(y)
        test_pred.append(batch_pred)
        test_logits.append(batch_logits)
    print("Test accuracy : {:.2f}".format(100*np.mean(test_acc))) 
    #val_pred = np.vstack(val_pred)
    #val_y = np.vstack(val_y)
    #print(mapk(val_y,val_pred))
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, num_layers))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

epoch 0: Training loss : 0.457426
Validation loss : 0.394261
epoch 1: Training loss : 0.306372
Validation loss : 0.218551
epoch 2: Training loss : 0.224340
Validation loss : 0.207967
epoch 3: Training loss : 0.201930
Validation loss : 0.181982
epoch 4: Training loss : 0.194980
Validation loss : 0.174965
epoch 5: Training loss : 0.188409
Validation loss : 0.183822
epoch 6: Training loss : 0.182877
Validation loss : 0.186902
epoch 7: Training loss : 0.176397
Validation loss : 0.148297
epoch 8: Training loss : 0.170958
Validation loss : 0.147477
epoch 9: Training loss : 0.173400
Validation loss : 0.158915
epoch 10: Training loss : 0.168600
Validation loss : 0.160263
epoch 11: Training loss : 0.166114
Validation loss : 0.139857
epoch 12: Training loss : 0.162067
Validation loss : 0.163470
epoch 13: Train

epoch 126: Training loss : 0.086018
Validation loss : 0.087669
epoch 127: Training loss : 0.082072
Validation loss : 0.088933
epoch 128: Training loss : 0.083393
Validation loss : 0.091763
epoch 129: Training loss : 0.082675
Validation loss : 0.088517
epoch 130: Training loss : 0.082241
Validation loss : 0.092250
epoch 131: Training loss : 0.081705
Validation loss : 0.088660
epoch 132: Training loss : 0.082244
Validation loss : 0.090968
epoch 133: Training loss : 0.082974
Validation loss : 0.091244
epoch 134: Training loss : 0.081624
Validation loss : 0.091822
epoch 135: Training loss : 0.082990
Validation loss : 0.108163
epoch 136: Training loss : 0.081735
Validation loss : 0.091542
epoch 137: Training loss : 0.079726
Validation loss : 0.102755
epoch 138: Training loss : 0.080370
Validation loss : 0.088619
epoch 139: Training loss : 0.079596
Validation loss : 0.092584
epoch 140: Training loss : 0.078831
Validation loss : 0.093349
epoch 141: Training loss : 0.078706
Validation loss : 0

In [34]:
y_ =[]
print(np.sum([np.sum(pred)for pred in val_pred]), np.sum([np.sum(pred)for pred in val_y]))
print(np.sum([np.sum(pred) for pred in test_pred]), np.sum([np.sum(pred) for pred in test_y]))

18545.0 20026.0
56 12263.0


In [36]:
val_y = np.concatenate(val_y)
test_y = np.concatenate(test_y)

ValueError: zero-dimensional arrays cannot be concatenated

In [37]:
from sklearn.metrics import confusion_matrix
tn,fp,fn,tp = confusion_matrix(val_y,val_pred).ravel()
(tp,fp,fn,tp)

(18421, 124, 1605, 18421)

In [38]:
DR = tp/(tp+fp)
FAR = fp/(tn+fp)
recall = tp/(tp+fn)
f1 = 2*(recall*DR)/(recall+DR)
(DR,FAR,recall,f1,np.mean(val_acc))

(0.9933135616069021,
 0.006208070491639131,
 0.9198541895535803,
 0.9551735760026964,
 0.95545)

In [39]:
tn,fp,fn,tp = confusion_matrix(test_y,np.concatenate(test_pred)).ravel()
DR = tp/(tp+fp)
FAR = fp/(tn+fp)
recall = tp/(tp+fn)
f1 = 2*(recall*DR)/(recall+DR)
(DR,FAR,recall,f1,np.mean(test_acc))


(0.30357142857142855,
 0.00023935631563119488,
 0.001386283943570089,
 0.0027599642828151634,
 0.9298801)

In [40]:
tn,fp,fn,tp = confusion_matrix(np.concatenate(train_label),np.concatenate(train_pred)).ravel()
DR = tp/(tp+fp)
FAR = fp/(tn+fp)
recall = tp/(tp+fn)
f1 = 2*(recall*DR)/(recall+DR)
(DR,FAR,recall,f1,np.mean(test_acc)), (tp,fp,fn,tn)

((0.9749405919205012,
  0.025056947608200455,
  0.9757646041355452,
  0.9753524239893904,
  0.9298801),
 (49643, 1276, 1233, 49648))

In [41]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(np.concatenate(train_logits,0),np.concatenate(train_label))

GaussianNB(priors=None)

In [42]:
val_pred = clf.predict(np.concatenate(val_logits,0))
tn,fp,fn,tp = confusion_matrix(y_valid,val_pred).ravel()
(tp,fp,fn,tp)

(18421, 124, 1605, 18421)

In [43]:
acc = (tp+tn)/(tp+tn+fp+fn)
DR = tp/(tp+fp)
FAR = fp/(tn+fp)
recall = tp/(tp+fn)
f1 = 2*(recall*DR)/(recall+DR)
(DR,FAR,recall,f1,acc)

(0.9933135616069021,
 0.006208070491639131,
 0.9198541895535803,
 0.9551735760026964,
 0.956775)

In [44]:
test_pred_ = clf.predict(np.concatenate(test_logits,0))
tn,fp,fn,tp = confusion_matrix(test_y,test_pred_).ravel()
acc = (tp+tn)/(tp+tn+fp+fn)
DR = tp/(tp+fp)
FAR = fp/(tn+fp)
recall = tp/(tp+fn)
f1 = 2*(recall*DR)/(recall+DR)
(DR,FAR,recall,f1,acc)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: RuntimeWarning: invalid value encountered in double_scalars


(0.0, 7.978543854373163e-05, 0.0, nan, 0.929931506849315)

In [45]:
train_pred_ = clf.predict(np.concatenate(train_logits,0))
tn,fp,fn,tp = confusion_matrix(np.concatenate(train_label),train_pred_).ravel()
acc = (tp+tn)/(tp+tn+fp+fn)
DR = tp/(tp+fp)
FAR = fp/(tn+fp)
recall = tp/(tp+fn)
f1 = 2*(recall*DR)/(recall+DR)
(DR,FAR,recall,f1,acc)


(0.9989462370123712,
 0.0009818553138009583,
 0.9316573630002358,
 0.9641291634884311,
 0.9653536345776031)